## This ipynb file is WIP, and might need to be part of the pipeline, its purpose is to scan the the big boy file to get enough minority classes combined with SMOTE to get a dataset that is not too imbalanced

In [ ]:
import pandas as pd

# Parameters
file_path = "normal.csv"  # Replace with your file path
column_name = "your_column"   # Replace with the column you're filtering on
value = True                  # The value to filter on (e.g., True)

# Read CSV in chunks to avoid memory issues
chunksize = 100000  # Adjust this if needed for larger files
filtered_rows = []

# Define the mapping of original Spanish column names to English column names
column_mapping = {
    "fecha_dato": "report_date",
    "ncodpers": "customer_id",
    "ind_empleado": "employee_index",
    "pais_residencia": "country_residence",
    "sexo": "gender",
    "age": "age",
    "fecha_alta": "contract_start_date",
    "ind_nuevo": "new_customer_index",
    "antiguedad": "seniority_months",
    "indrel": "primary_customer_status",
    "ult_fec_cli_1t": "last_primary_customer_date",
    "indrel_1mes": "customer_type_start_month",
    "tiprel_1mes": "customer_relation_type",
    "indresi": "residence_index",
    "indext": "foreigner_index",
    "conyuemp": "spouse_employee_index",
    "canal_entrada": "join_channel",
    "indfall": "deceased_index",
    "tipodom": "address_type",
    "cod_prov": "province_code",
    "nomprov": "province_name",
    "ind_actividad_cliente": "activity_index",
    "renta": "gross_income",
    "segmento": "customer_segment",
    "ind_ahor_fin_ult1": "saving_account",
    "ind_aval_fin_ult1": "guarantee",
    "ind_cco_fin_ult1": "current_account",
    "ind_cder_fin_ult1": "derivada_account",
    "ind_cno_fin_ult1": "payroll_account",
    "ind_ctju_fin_ult1": "junior_account",
    "ind_ctma_fin_ult1": "more_particular_account",
    "ind_ctop_fin_ult1": "particular_account",
    "ind_ctpp_fin_ult1": "particular_plus_account",
    "ind_deco_fin_ult1": "short_term_deposits",
    "ind_deme_fin_ult1": "medium_term_deposits",
    "ind_dela_fin_ult1": "long_term_deposits",
    "ind_ecue_fin_ult1": "e_account",
    "ind_fond_fin_ult1": "funds",
    "ind_hip_fin_ult1": "mortgage",
    "ind_plan_fin_ult1": "pensions",
    "ind_pres_fin_ult1": "loans",
    "ind_reca_fin_ult1": "taxes",
    "ind_tjcr_fin_ult1": "credit_card",
    "ind_valo_fin_ult1": "securities",
    "ind_viv_fin_ult1": "home_account",
    "ind_nomina_ult1": "payroll",
    "ind_nom_pens_ult1": "pensions_payments",
    "ind_recibo_ult1": "direct_debit"
}

# Iterate through the file in chunks
for chunk in pd.read_csv(file_path, chunksize=chunksize):
    # Filter rows where the column equals the desired value
    matching_rows = chunk[chunk[column_name] == value]
    filtered_rows.append(matching_rows)
    # Rename the columns to english
    chunk.rename(columns=column_mapping, inplace=True)

# Define new column names
fixed_deposits_col = 'fixed_deposits'
loan_col = 'loan'
credit_card_debit_card_col = 'credit_card_debit_card'
account_col = 'account'

# Check and create a new column for fixed deposits, if it doesn't exist
if fixed_deposits_col not in df.columns:
    deposit_columns = [
        "short_term_deposits",  # ind_deco_fin_ult1
        "medium_term_deposits",  # ind_deme_fin_ult1
        "long_term_deposits"    # ind_dela_fin_ult1
    ]
    df[fixed_deposits_col] = df[deposit_columns].any(axis=1).astype(int)

# Check and create a new column for loans, if it doesn't exist
if loan_col not in df.columns:
    loan_columns = [
        "loans",                # ind_pres_fin_ult1
        "pensions"             # ind_plan_fin_ult1
    ]
    df[loan_col] = df[loan_columns].any(axis=1).astype(int)

# Check and create a new column for credit and debit cards, if it doesn't exist
if credit_card_debit_card_col not in df.columns:
    credit_card_columns = [
        "credit_card",         # ind_tjcr_fin_ult1
        "direct_debit"        # ind_recibo_ult1
    ]
    df[credit_card_debit_card_col] = df[credit_card_columns].any(axis=1).astype(int)

# Check and create a new column for all accounts combined, if it doesn't exist
if account_col not in df.columns:
    account_columns = [
        "saving_account",      # ind_ahor_fin_ult1
        "current_account",     # ind_cco_fin_ult1
        "derivada_account",    # ind_cder_fin_ult1
        "payroll_account",     # ind_cno_fin_ult1
        "junior_account",      # ind_ctju_fin_ult1
        "more_particular_account",  # ind_ctma_fin_ult1
        "particular_account",   # ind_ctop_fin_ult1
        "particular_plus_account", # ind_ctpp_fin_ult1
        "e_account",           # ind_ecue_fin_ult1
        "funds",               # ind_fond_fin_ult1
        "home_account",        # ind_viv_fin_ult1
    ]
    df[account_col] = df[account_columns].any(axis=1).astype(int)
    
    # List of columns to drop, we merged these columns for to keep it simple to present 
    # The model used for commercial purposes does not merge the products together
    columns_to_drop = [
        'saving_account', 'guarantee', 'current_account', 'derivada_account', 'payroll_account', 
        'junior_account', 'more_particular_account', 'particular_account', 'particular_plus_account', 
        'short_term_deposits', 'medium_term_deposits', 'long_term_deposits', 'e_account', 'funds', 
        'mortgage', 'pensions', 'loans', 'taxes', 'credit_card', 'securities', 'home_account', 
         'payroll', 'pensions_payments', 'direct_debit'
    ]

    # Drop the columns if they exist
    df = chunk.drop(columns=[col for col in columns_to_drop if col in df.columns])
    # Stop if we have enough rows
if sum(len(rows) for rows in filtered_rows) >= 5000:
    break

# Combine filtered chunks into one DataFrame
result = pd.concat(filtered_rows).head(5000)

# Save the result to a new CSV file
result.to_csv("subset.csv", index=False)

print(f"Saved {len(result)} rows to 'subset.csv'")


In [3]:
import pandas as pd

# Load the CSV file
file_path = "cleaned_covid.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Select the last 4 columns
last_4_columns = df.iloc[:, -4:]

# Display class distribution for each of the last 4 columns
for column in last_4_columns.columns:
    print(f"\nClass distribution for '{column}':")
    print(last_4_columns[column].value_counts())
    print(last_4_columns[column].value_counts(normalize=True) * 100)

# Optional: Save the distribution to a CSV file
distribution = last_4_columns.apply(lambda x: x.value_counts())
distribution.to_csv("class_imbalance_counts.csv")





Class distribution for 'fixed_deposits':
fixed_deposits
0    9879
1      67
Name: count, dtype: int64
fixed_deposits
0    99.326362
1     0.673638
Name: proportion, dtype: float64

Class distribution for 'loan':
loan
0    9938
1       8
Name: count, dtype: int64
loan
0    99.919566
1     0.080434
Name: proportion, dtype: float64

Class distribution for 'credit_card_debit_card':
credit_card_debit_card
0    9266
1     680
Name: count, dtype: int64
credit_card_debit_card
0    93.163081
1     6.836919
Name: proportion, dtype: float64

Class distribution for 'account':
account
1    9850
0      96
Name: count, dtype: int64
account
1    99.034788
0     0.965212
Name: proportion, dtype: float64
